In [59]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [60]:
data = pd.read_csv('/Users/sa12/Documents/Repositories/The-CyberChase/DATA/Clean4Model')
data = data.drop(columns=['Unnamed: 0','Location_China','Location_France','Location_Germany','Location_Hong Kong','Location_Iran','Location_Italy','Location_Japan','Location_Singapore','Location_United Kingdom','Location_United States'])

In [61]:
# Features and target
X = data.drop(columns=['Country'])
y = data['Country']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [63]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [64]:
model = LogisticRegression(
    multi_class='multinomial',
    solver='lbfgs',
    max_iter=1000,
    random_state=42
)
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.5936662551320313


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Classification Report:
               precision    recall  f1-score   support

           1       0.82      1.00      0.90    106603
           2       0.40      0.73      0.52     66350
           3       0.26      0.10      0.14     38184
           4       0.57      0.71      0.64     34126
           5       0.00      0.00      0.00     17413
           6       0.37      0.00      0.00     15252
           7       0.00      0.00      0.00      9433
           8       0.00      0.00      0.00      7687
           9       0.00      0.00      0.00      7009
          10       0.00      0.00      0.00      6544

    accuracy                           0.59    308601
   macro avg       0.24      0.25      0.22    308601
weighted avg       0.48      0.59      0.51    308601


Confusion Matrix:
 [[106451    132      0     19      0      1      0      0      0      0]
 [ 10233  48709   1819   5547     30      7      1      1      0      3]
 [  1894  25562   3681   7031      7      8      0

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
